In [4]:
conda install importlib_metadata

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/odbiz

  added / updated specs:
    - importlib_metadata


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    importlib-metadata-4.11.4  |   py38h578d9bd_0          33 KB  http://jfrog-platform-artifactory-ha.jfrog-system:8081/artifactory/api/conda/conda-forge-remote
    importlib_metadata-4.11.4  |       hd8ed1ab_0           4 KB  http://jfrog-platform-artifactory-ha.jfrog-system:8081/artifactory/api/conda/conda-forge-remote
    zipp-3.8.1                 |     pyhd8ed1ab_0          13 KB  http://jfrog-platform-artifactory-ha.jfrog-system:8081/artifactory/api/conda/conda-forge-remote
    ------------------------------------------------------------
                                           Total:          49 KB

The following NEW packages will be INSTALLED:

  importlib-metadata artifactory/api/conda/con

In [2]:
conda install rl_helper

Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: failed with initial frozen solve. Retrying with flexible solve.

PackagesNotFoundError: The following packages are not available from current channels:

  - rl_helper

Current channels:

  - http://jfrog-platform-artifactory-ha.jfrog-system:8081/artifactory/api/conda/conda-forge-remote/linux-64
  - http://jfrog-platform-artifactory-ha.jfrog-system:8081/artifactory/api/conda/conda-forge-remote/noarch

To search for alternate channels that may provide the conda package you're
looking for, navigate to

    https://anaconda.org

and use the search bar at the top of the page.



Note: you may need to restart the kernel to use updated packages.


In [2]:

#1_create_pairs




import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import json
import pandas as pd
import re
from rl_helper import strip_accents, AddressClean, haversine
import recordlinkage as rl
from recordlinkage.preprocessing import clean
import numpy as np
import math





sourcefile = '/home/jovyan/ODBiz/7-Deduplication/inputs/1NAICS_Test.json'
with open(sourcefile) as source_f:
    Source = json.load(source_f)
    

df = pd.read_csv(Source["filename"],
                 encoding=Source["encoding"],
                 index_col=Source["index"])

#df = pd.read_csv( '/home/jovyan/ODBiz/Deduplication/ODHF_copy/inputs/1NAICS_Test.json', encoding=Source["encoding"], index_col=Source["index"])

print('I. Preprocessing - renaming columns, removing accents, and making string replacements.')




# reduce database to only the columns we use for comparisons
df = df[Source["column_map"].values()]
column_map = {val: key for key, val in Source["column_map"].items()}
df = df.rename(columns = column_map)

# remove accents

text_cols = ['Name','Address','StreetName','City']

for col in text_cols:
    df.loc[~df[col].isnull(),col]=df.loc[~df[col].isnull(),col].apply(strip_accents)
    
#make names lowercase
df['Name'] = df['Name'].apply(str)
df['Name'] = df['Name'].str.lower()

df['Address'] = df['Address'].apply(str)
df['Address'] = df['Address'].str.lower()

df['StreetName'] = df['StreetName'].apply(str)
df['StreetName'] = df['StreetName'].str.lower()

#standardise addresses using "AddressClean" function in the rl_helper module

df.loc[(~df['StreetName'].isnull()) & (df.Province != 'qc'), 'StreetName'] = df.loc[(~df['StreetName'].isnull()) & (df.Province != 'qc'), 'StreetName'].apply(AddressClean, args = ('en',))
df.loc[(~df['StreetName'].isnull()) & (df.Province == 'qc'), 'StreetName'] = df.loc[(~df['StreetName'].isnull()) & (df.Province == 'qc'), 'StreetName'].apply(AddressClean, args = ('fr',))

df.loc[(~df['Address'].isnull()) & (df.Province != 'qc'), 'Address'] = df.loc[(~df['Address'].isnull()) & (df.Province != 'qc'), 'Address'].apply(AddressClean, args = ('en',))
df.loc[(~df['Address'].isnull()) & (df.Province == 'qc'), 'Address'] = df.loc[(~df['Address'].isnull()) & (df.Province == 'qc'), 'Address'].apply(AddressClean, args = ('fr',))

#remove periods, apostrophes, commas, and hypens in the Name and address columns

r_list = [r".", r",", r"'", r"-"]

for r in r_list:

    df["Name"] = df["Name"].str.replace(r, ' ', regex=False)
    df["Address"] = df["Address"].str.replace(r, ' ', regex=False)

#remove excess whitespace
df["Name"] = df["Name"].str.replace(r" +", " ", regex=True)
df["Address"] = df["Address"].str.replace(r" +", " ", regex=True)

#standardise postal codes - just remove empty space and make sure it's all lower case

df.loc[~df.PostalCode.isnull(), 'PostalCode'] = df.loc[~df.PostalCode.isnull(), 'PostalCode'].str.replace(' ', '', regex=True).str.lower()

#create an extra temporary Name column with an additional level of cleaning

df['NameClean'] = clean(df["Name"])

#Some records have street number and street name, but no address field filled

df.loc[(df.Address.isnull())&\
       (~df.StreetName.isnull()),'Address']\
    =clean(df.loc[(df.Address.isnull())&\
       (~df.StreetName.isnull()),'StreetNumber']+' '+\
           df.loc[(df.Address.isnull())&\
       (~df.StreetName.isnull()),'StreetName']+' '+\
        df.loc[(df.Address.isnull())&\
       (~df.StreetName.isnull()),'City'])

#df.to_csv('NEW.csv')

r"""
II. Record Linkage


This is the section that uses the record linkage package to determine candidate pairs,
which will be evaluated separately.
"""
print('II. Record linkage - Now creating multiindex and performing comparisons')

indexer = rl.Index()
indexer.block('PostalCode')
candidate_links = indexer.index(df)

print('Computing metrics for {} candidate pairs'.format(len(candidate_links)))

# likely to be a lot of records to match, so split into chunks
n = math.ceil(len(candidate_links) / 1E5)
chunks = rl.index_split(candidate_links, n)

# Comparison step
results = []

# n_jobs specifies number of cores for running in parallel
compare = rl.Compare(n_jobs=4)

compare.exact('StreetNumber', 'StreetNumber', label='StrNum_Match')
compare.exact('PostalCode', 'PostalCode', label='PC_Match')
compare.exact('FileName', 'FileName', label='File_Match')
#compare.exact('Type', 'Type', label='Type_Match')
compare.string('Address', 'Address', method='damerau_levenshtein', label='Addr_DL')
compare.string('Address', 'Address', method='cosine', label='Addr_CS')
compare.string('Address', 'Address', method='damerau_levenshtein', label='StrName_DL')
compare.string('Address', 'Address', method='cosine', label='StrName_CS')
compare.string('City', 'City', method='damerau_levenshtein', label='City_DL')
compare.string('Name', 'Name', method='damerau_levenshtein', label='Name_DL')
compare.string('Name', 'Name', method='cosine', label='Name_CS')
compare.string('Name', 'Name', method='qgram', label='Name_Q')
compare.string("NameClean", "NameClean", method='damerau_levenshtein', label="CleanName_DL")


i = 0
for chunk in chunks:
    i += 1
    print('processing chunk {} of {}'.format(i,n))

    features = compare.compute(chunk, df)

    #reduce comparison matrix to entries where the name score is reasonably high

    cutoff = 0
    features = features.loc[features.Name_CS > cutoff]
    results.append(features)
f = pd.concat(results)
print('Score cut-off of {} reduced candidate pairs to {}'.format(cutoff, len(f)))



f['idx1'] = f.index.get_level_values(0)
f['idx2'] = f.index.get_level_values(1)

print('Merging on original dataframe and computing distance.')
f=f.merge(df, left_on='idx1', how='left', right_on='idx')

f=f.merge(df, left_on='idx2', how='left', right_on='idx', suffixes=('_1','_2'))

#add Haversine distance to pairs

f['Distance']=np.nan
f[['Latitude_1', 'Latitude_2', 'Longitude_1', 'Longitude_2']] = f[['Latitude_1', 'Latitude_2', 'Longitude_1', 'Longitude_2']].astype(float)
f.loc[(~f.Latitude_1.isnull())&(~f.Latitude_2.isnull()),'Distance']=f.loc[(~f.Latitude_1.isnull())&(~f.Latitude_2.isnull())].apply(lambda row: haversine(row), axis=1)

f=f[['idx1',
     'idx2',
     'FileName_1',
     'FileName_2',
     'File_Match',
     'Name_1',
     'Name_2',
     'Name_DL',
     'Name_CS',
     'Name_Q',
     'CleanName_DL',
     'Address_1',
     'Address_2',
     'Addr_DL',
     'Addr_CS',
     'StrNum_Match',
     'StrName_DL',
     'StrName_CS',
     'PostalCode_1',
     'PostalCode_2',
     'PC_Match',
     'City_1',
     'City_2',
     'City_DL',
     'CSDUID_1',
     'CSDUID_2',
     'Distance']]


f.to_csv('outputs/pairs_PC.csv'.format(Source["output_name"]),index=False,encoding='cp1252')

#output pairs that have addresses and coordinates separately from those missing one or more addresses/coordinates
f.loc[(~f.Distance.isnull())&(~f.Address_1.isnull())&(~f.Address_2.isnull())].to_csv('outputs/FullInfoPC.csv'.format(Source["output_name"]),index=False,encoding='cp1252')
f.loc[(f.Distance.isnull())|(f.Address_1.isnull())|(f.Address_2.isnull())].to_csv('outputs/PartialInfoPC.csv'.format(Source["output_name"]),index=False,encoding='cp1252')


I. Preprocessing - renaming columns, removing accents, and making string replacements.
II. Record linkage - Now creating multiindex and performing comparisons
Computing metrics for 64 candidate pairs
processing chunk 1 of 1
Score cut-off of 0 reduced candidate pairs to 59
Merging on original dataframe and computing distance.


In [18]:

# 4_creat _pairs_csd 



# -*- coding: utf-8 -*-
r"""
This script looks for potential duplicates in a database

We are assuming a set of standard columns,
Name, Address, StreetNumber, StreetName, City, PostalCode, Province, Latitude, Longitude

There are two sections, an initial Preprocessing step, and then the comparison script

With minor modification, this can also be used for record linkage between two files.

I) Preprocessing:
    0. Read in JSON 'source' file that contains
        i. file name of database
        ii. mappings from database column names to standard names
        iii. a short dictionary of terms to replace in the Name field to improve
            potential matches (e.g., 'ch' for 'centre hospitalier')
    1. Read in database (columns as values from json)
    2. Strip all accents from all text fields
    3. Process Address and Street Name fields to standardise street types
    4. run recordlinkage's "clean" function to remove extra whitespace and any
        remaining non-ascii characters, and anything in parentheses
    5. Standardise PostalCode
        
II) Record Linkage:
    Use RecordLinkageToolkit to perform comparisons and create index pairs:
        Criteria:
                Province - Block (consider only matches where equal)
                Name - Damerau-Levenshtein, qgram
                Address - Damerau-Levenshtein, Cosine
                StreetNumber - Exact
                StreetName - Damerau-Levenshtein, Cosine
                City - Damerau-Levenshtein
                PostalCode - Exact
                Latitude/Longitude - Distance

    The result is a Pandas multiindex object, which we then use to create a file
    where every line contains the two objects being compared. 
    
    This output file will be fed to a machine learning script for classification.
"""

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import json
import pandas as pd
import re
from rl_helper import strip_accents, AddressClean, haversine
import recordlinkage as rl
from recordlinkage.preprocessing import clean
import numpy as np
import math

'''
Read in source file, data file, and rename data file columns
'''
sourcefile = "/home/jovyan/ODBiz/Deduplication/ODHF_copy/inputs/deduplicated_PC.json"
with open(sourcefile) as source_f:
    Source = json.load(source_f)
    

df = pd.read_csv(Source["filename"],
               encoding=Source["encoding"],
               index_col=Source["index"])

print('I. Preprocessing - renaming columns, removing accents, and making string replacements.')

# reduce database to only the columns we use for comparisons
df = df[Source["column_map"].values()]
column_map = {val: key for key, val in Source["column_map"].items()}
df = df.rename(columns = column_map)

# remove accents

text_cols = ['Name','Address','StreetName','City']

for col in text_cols:
    df.loc[~df[col].isnull(),col]=df.loc[~df[col].isnull(),col].apply(strip_accents)
    
#make names lowercase
df['Name'] = df['Name'].apply(str)
df['Name'] = df['Name'].str.lower()

df['Address'] = df['Address'].apply(str)
df['Address'] = df['Address'].str.lower()

df['StreetName'] = df['StreetName'].apply(str)
df['StreetName'] = df['StreetName'].str.lower()

#apply text swaps in the Name column    
#for swap in Source["text_map"]:
    #start = r'\b'+re.escape(swap[0])+r'\b'
    #df["Name"] = df["Name"].str.replace(start,swap[1], regex=True)




#standardise addresses using "AddressClean" function in the rl_helper module

df.loc[(~df['StreetName'].isnull()) & (df.Province != 'qc'), 'StreetName'] = df.loc[(~df['StreetName'].isnull()) & (df.Province != 'qc'), 'StreetName'].apply(AddressClean, args = ('en',))
df.loc[(~df['StreetName'].isnull()) & (df.Province == 'qc'), 'StreetName'] = df.loc[(~df['StreetName'].isnull()) & (df.Province == 'qc'), 'StreetName'].apply(AddressClean, args = ('fr',))

df.loc[(~df['Address'].isnull()) & (df.Province != 'qc'), 'Address'] = df.loc[(~df['Address'].isnull()) & (df.Province != 'qc'), 'Address'].apply(AddressClean, args = ('en',))
df.loc[(~df['Address'].isnull()) & (df.Province == 'qc'), 'Address'] = df.loc[(~df['Address'].isnull()) & (df.Province == 'qc'), 'Address'].apply(AddressClean, args = ('fr',))

#remove periods, apostrophes, commas, and hypens in the Name and address columns

r_list = [r".", r",", r"'", r"-"]

for r in r_list:

    df["Name"] = df["Name"].str.replace(r, ' ', regex=False)
    df["Address"] = df["Address"].str.replace(r, ' ', regex=False)

#remove excess whitespace
df["Name"] = df["Name"].str.replace(r" +", " ", regex=True)
df["Address"] = df["Address"].str.replace(r" +", " ", regex=True)

#standardise postal codes - just remove empty space and make sure it's all lower case

df.loc[~df.PostalCode.isnull(), 'PostalCode'] = df.loc[~df.PostalCode.isnull(), 'PostalCode'].str.replace(' ', '', regex=True).str.lower()

#create an extra temporary Name column with an additional level of cleaning

df['NameClean'] = clean(df["Name"])

#Some records have street number and street name, but no address field filled

df.loc[(df.Address.isnull())&\
       (~df.StreetName.isnull()),'Address']\
    =clean(df.loc[(df.Address.isnull())&\
       (~df.StreetName.isnull()),'StreetNumber']+' '+\
           df.loc[(df.Address.isnull())&\
       (~df.StreetName.isnull()),'StreetName']+' '+\
        df.loc[(df.Address.isnull())&\
       (~df.StreetName.isnull()),'City'])


r"""
II. Record Linkage


This is the section that uses the record linkage package to determine candidate pairs,
which will be evaluated separately.
"""
print('II. Record linkage - Now creating multiindex and performing comparisons')

indexer = rl.Index()
indexer.block('CSDUID')
candidate_links = indexer.index(df)

print('Computing metrics for {} candidate pairs'.format(len(candidate_links)))

# likely to be a lot of records to match, so split into chunks
n = math.ceil(len(candidate_links) / 1E5)
chunks = rl.index_split(candidate_links, n)

# Comparison step
results = []

# n_jobs specifies number of cores for running in parallel
compare = rl.Compare(n_jobs=4)

compare.exact('StreetNumber', 'StreetNumber', label='StrNum_Match')
compare.exact('PostalCode', 'PostalCode', label='PC_Match')
compare.exact('FileName', 'FileName', label='File_Match')
#compare.exact('Type', 'Type', label='Type_Match')
compare.string('Address', 'Address', method='damerau_levenshtein', label='Addr_DL')
compare.string('Address', 'Address', method='cosine', label='Addr_CS')
compare.string('Address', 'Address', method='damerau_levenshtein', label='StrName_DL')
compare.string('Address', 'Address', method='cosine', label='StrName_CS')
compare.string('City', 'City', method='damerau_levenshtein', label='City_DL')
compare.string('Name', 'Name', method='damerau_levenshtein', label='Name_DL')
compare.string('Name', 'Name', method='cosine', label='Name_CS')
compare.string('Name', 'Name', method='qgram', label='Name_Q')
compare.string("NameClean", "NameClean", method='damerau_levenshtein', label="CleanName_DL")


i = 0
for chunk in chunks:
    i += 1
    print('processing chunk {} of {}'.format(i,n))

    features = compare.compute(chunk, df)

    #reduce comparison matrix to entries where the name score is reasonably high

    cutoff = 0.3
    features = features.loc[features.Name_CS > cutoff]
    results.append(features)
f = pd.concat(results)
print('Score cut-off of {} reduced candidate pairs to {}'.format(cutoff, len(f)))

f['idx1'] = f.index.get_level_values(0)
f['idx2'] = f.index.get_level_values(1)

print('Merging on original dataframe and computing distance.')
f=f.merge(df, left_on='idx1', how='left', right_on='idx')

f=f.merge(df, left_on='idx2', how='left', right_on='idx', suffixes=('_1','_2'))

#add Haversine distance to pairs

f['Distance']=np.nan
f[['Latitude_1', 'Latitude_2', 'Longitude_1', 'Longitude_2']] = f[['Latitude_1', 'Latitude_2', 'Longitude_1', 'Longitude_2']].astype(float)
f.loc[(~f.Latitude_1.isnull())&(~f.Latitude_2.isnull()),'Distance']=f.loc[(~f.Latitude_1.isnull())&(~f.Latitude_2.isnull())].apply(lambda row: haversine(row), axis=1)

f=f[['idx1',
     'idx2',
     'FileName_1',
     'FileName_2',
     'File_Match',
     'Name_1',
     'Name_2',
     'Name_DL',
     'Name_CS',
     'Name_Q',
     'CleanName_DL',
     'Address_1',
     'Address_2',
     'Addr_DL',
     'Addr_CS',
     'StrNum_Match',
     'StrName_DL',
     'StrName_CS',
     'PostalCode_1',
     'PostalCode_2',
     'PC_Match',
     'City_1',
     'City_2',
     'City_DL',
     'CSDUID_1',
     'CSDUID_2',
     'Distance']]


f.to_csv('outputs/pairs_CSD.csv'.format(Source["output_name"]),index=False,encoding='cp1252')

I. Preprocessing - renaming columns, removing accents, and making string replacements.
II. Record linkage - Now creating multiindex and performing comparisons
Computing metrics for 80995 candidate pairs
processing chunk 1 of 1
Score cut-off of 0.3 reduced candidate pairs to 5695
Merging on original dataframe and computing distance.


In [20]:

# 7_create_pairs_cihi



# -*- coding: utf-8 -*-
r"""
This script looks for potential duplicates in a database

We are assuming a set of standard columns,
Name, Address, StreetNumber, StreetName, City, PostalCode, Province, Latitude, Longitude

There are two sections, an initial Preprocessing step, and then the comparison script

With minor modification, this can also be used for record linkage between two files.

I) Preprocessing:
    0. Read in JSON 'source' file that contains
        i. file name of database
        ii. mappings from database column names to standard names
        iii. a short dictionary of terms to replace in the Name field to improve
            potential matches (e.g., 'ch' for 'centre hospitalier')
    1. Read in database (columns as values from json)
    2. Strip all accents from all text fields
    3. Process Address and Street Name fields to standardise street types
    4. run recordlinkage's "clean" function to remove extra whitespace and any
        remaining non-ascii characters, and anything in parentheses
    5. Standardise PostalCode
        
II) Record Linkage:
    Use RecordLinkageToolkit to perform comparisons and create index pairs:
        Criteria:
                Province - Block (consider only matches where equal)
                Name - Damerau-Levenshtein, qgram
                Address - Damerau-Levenshtein, Cosine
                StreetNumber - Exact
                StreetName - Damerau-Levenshtein, Cosine
                City - Damerau-Levenshtein
                PostalCode - Exact
                Latitude/Longitude - Distance

    The result is a Pandas multiindex object, which we then use to create a file
    where every line contains the two objects being compared. 
    
    This output file will be fed to a machine learning script for classification.
"""

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import json
import pandas as pd
import re
from rl_helper import strip_accents, AddressClean, haversine
import recordlinkage as rl
from recordlinkage.preprocessing import clean
import numpy as np
import math

'''
Read in source file, data file, and rename data file columns
'''
sourcefile = "/home/jovyan/ODBiz/Deduplication/ODHF_copy/inputs/deduplicated_CSD.json"
with open(sourcefile) as source_f:
    Source = json.load(source_f)
    

df = pd.read_csv(Source["filename"],
               encoding=Source["encoding"],
               index_col=Source["index"])

print('I. Preprocessing - renaming columns, removing accents, and making string replacements.')

# reduce database to only the columns we use for comparisons
df = df[Source["column_map"].values()]
column_map = {val: key for key, val in Source["column_map"].items()}
df = df.rename(columns = column_map)

# remove accents

text_cols = ['Name','Address','StreetName','City']

for col in text_cols:
    df.loc[~df[col].isnull(),col]=df.loc[~df[col].isnull(),col].apply(strip_accents)
    
#make names and addresses lowercase
df['Name'] = df['Name'].apply(str)
df['Name'] = df['Name'].str.lower()

df['Address'] = df['Address'].apply(str)
df['Address'] = df['Address'].str.lower()

df['StreetName'] = df['StreetName'].apply(str)
df['StreetName'] = df['StreetName'].str.lower()



#standardise addresses using "AddressClean" function in the rl_helper module

df.loc[(~df['StreetName'].isnull()) & (df.Province != 'qc'), 'StreetName'] = df.loc[(~df['StreetName'].isnull()) & (df.Province != 'qc'), 'StreetName'].apply(AddressClean, args = ('en',))
df.loc[(~df['StreetName'].isnull()) & (df.Province == 'qc'), 'StreetName'] = df.loc[(~df['StreetName'].isnull()) & (df.Province == 'qc'), 'StreetName'].apply(AddressClean, args = ('fr',))

df.loc[(~df['Address'].isnull()) & (df.Province != 'qc'), 'Address'] = df.loc[(~df['Address'].isnull()) & (df.Province != 'qc'), 'Address'].apply(AddressClean, args = ('en',))
df.loc[(~df['Address'].isnull()) & (df.Province == 'qc'), 'Address'] = df.loc[(~df['Address'].isnull()) & (df.Province == 'qc'), 'Address'].apply(AddressClean, args = ('fr',))

#remove periods, apostrophes, commas, and hypens in the Name and address columns

r_list = [r".", r",", r"'", r"-"]

for r in r_list:

    df["Name"] = df["Name"].str.replace(r, ' ', regex=False)
    df["Address"] = df["Address"].str.replace(r, ' ', regex=False)

#remove excess whitespace
df["Name"] = df["Name"].str.replace(r" +", " ", regex=True)
df["Address"] = df["Address"].str.replace(r" +", " ", regex=True)

#standardise postal codes - just remove empty space and make sure it's all lower case

df.loc[~df.PostalCode.isnull(), 'PostalCode'] = df.loc[~df.PostalCode.isnull(), 'PostalCode'].str.replace(' ', '', regex=True).str.lower()

#create an extra temporary Name column with an additional level of cleaning

df['NameClean'] = clean(df["Name"])

#Some records have street number and street name, but no address field filled

df.loc[(df.Address.isnull())&\
       (~df.StreetName.isnull()),'Address']\
    =clean(df.loc[(df.Address.isnull())&\
       (~df.StreetName.isnull()),'StreetNumber']+' '+\
           df.loc[(df.Address.isnull())&\
       (~df.StreetName.isnull()),'StreetName']+' '+\
        df.loc[(df.Address.isnull())&\
       (~df.StreetName.isnull()),'City'])

#df.to_csv('test.csv')

r"""
II. Record Linkage


This is the section that uses the record linkage package to determine candidate pairs,
which will be evaluated separately.
"""
print('II. Record linkage - Now creating multiindex and performing comparisons')

indexer = rl.Index()
indexer.block('Province')
candidate_links = indexer.index(df)

print('Computing metrics for {} candidate pairs'.format(len(candidate_links)))

# likely to be a lot of records to match, so split into chunks
n = math.ceil(len(candidate_links) / 1E5)
chunks = rl.index_split(candidate_links, n)

# Comparison step
results = []

# n_jobs specifies number of cores for running in parallel
compare = rl.Compare(n_jobs=4)

compare.exact('StreetNumber', 'StreetNumber', label='StrNum_Match')
compare.exact('PostalCode', 'PostalCode', label='PC_Match')
compare.exact('FileName', 'FileName', label='File_Match')
#compare.exact('Type', 'Type', label='Type_Match')
compare.string('Address', 'Address', method='damerau_levenshtein', label='Addr_DL')
compare.string('Address', 'Address', method='cosine', label='Addr_CS')
compare.string('Address', 'Address', method='damerau_levenshtein', label='StrName_DL')
compare.string('Address', 'Address', method='cosine', label='StrName_CS')
compare.string('City', 'City', method='damerau_levenshtein', label='City_DL')
compare.string('Name', 'Name', method='damerau_levenshtein', label='Name_DL')
compare.string('Name', 'Name', method='cosine', label='Name_CS')
compare.string('Name', 'Name', method='qgram', label='Name_Q')
compare.string("NameClean", "NameClean", method='damerau_levenshtein', label="CleanName_DL")


i = 0
for chunk in chunks:
    i += 1
    print('processing chunk {} of {}'.format(i,n))

    features = compare.compute(chunk, df)

    #reduce comparison matrix to entries where the name score is reasonably high

    cutoff = 0.5
    features = features.loc[features.Name_CS > cutoff]
    results.append(features)
f = pd.concat(results)
print('Score cut-off of {} reduced candidate pairs to {}'.format(cutoff, len(f)))

f['idx1'] = f.index.get_level_values(0)
f['idx2'] = f.index.get_level_values(1)

print('Merging on original dataframe and computing distance.')
f=f.merge(df, left_on='idx1', how='left', right_on='idx')

f=f.merge(df, left_on='idx2', how='left', right_on='idx', suffixes=('_1','_2'))

#add Haversine distance to pairs

f['Distance']=np.nan
f[['Latitude_1', 'Latitude_2', 'Longitude_1', 'Longitude_2']] = f[['Latitude_1', 'Latitude_2', 'Longitude_1', 'Longitude_2']].astype(float)
f.loc[(~f.Latitude_1.isnull())&(~f.Latitude_2.isnull()),'Distance']=f.loc[(~f.Latitude_1.isnull())&(~f.Latitude_2.isnull())].apply(lambda row: haversine(row), axis=1)

f=f[['idx1',
     'idx2',
     'FileName_1',
     'FileName_2',
     'File_Match',
     'Name_1',
     'Name_2',
     'Name_DL',
     'Name_CS',
     'Name_Q',
     'CleanName_DL',
     'Address_1',
     'Address_2',
     'Addr_DL',
     'Addr_CS',
     'StrNum_Match',
     'StrName_DL',
     'StrName_CS',
     'PostalCode_1',
     'PostalCode_2',
     'PC_Match',
     'City_1',
     'City_2',
     'City_DL',
     'CSDUID_1',
     'CSDUID_2',
     'Distance']]



f.to_csv('outputs/pairs_CIHI.csv'.format(Source["output_name"]),index=False,encoding='cp1252')


I. Preprocessing - renaming columns, removing accents, and making string replacements.
II. Record linkage - Now creating multiindex and performing comparisons
Computing metrics for 199609 candidate pairs
processing chunk 1 of 2
processing chunk 2 of 2
Score cut-off of 0.5 reduced candidate pairs to 745
Merging on original dataframe and computing distance.


In [ ]:

#8_scratch_duplicates_cihi


import pandas as pd

df = pd.read_csv('outputs/pairs_CIHI.csv')
final = pd.read_csv("inputs/deduplicated_CSD.csv", low_memory=False)


# CIHI_healthcare_facilities.csv

df = df[(df['FileName_1']  == "CIHI_healthcare_facilities.csv") | (df['FileName_2']  == "CIHI_healthcare_facilities.csv" )]
df.to_csv('outputs/pairs_CIHI.csv', index=False)


def hierarchy(id1, id2): 

#    try:

    check_file1 = final.loc[final.idx == id1].filename.item()
    check_file2 = final.loc[final.idx == id2].filename.item()
    check_name1 = final.loc[final.idx == id1].facility_name.item()
    check_name2 = final.loc[final.idx == id2].facility_name.item()


    if check_file1 == 'CIHI_healthcare_facilities.csv' and check_file2 != 'CIHI_healthcare_facilities.csv':
        return id1

    elif check_file2 == 'CIHI_healthcare_facilities.csv' and check_file1 != 'CIHI_healthcare_facilities.csv':
        return id2

    else:
        return id1

#    except ValueError:
 #       return "Item not found."

df["to_remove"] = df.apply(lambda x: hierarchy( id1 = x.idx1, id2 = x.idx2), axis = 1)





def check_1(name, distance, postalcode):

    if distance != None:
        if name > 0.5 and distance < 1 and postalcode == 1:
            return True
        else:
            return False

    else:
        return False

df["Check_1"] = df.apply(lambda x: check_1(name = x.Name_CS, distance = x.Distance, postalcode = x.PC_Match), axis=1)




def check_2(name, distance, postalcode):

    if distance != None:
        if name > 0.85 and distance < 5 and postalcode == 1:
            return True
        else:
            return False

    else:
        return False

df["Check_2"] = df.apply(lambda x: check_2(name = x.Name_CS, distance = x.Distance, postalcode = x.PC_Match), axis=1)




def check_3(name, distance):

    if distance != None:
        if name > 0.95 and distance < 5:
            return True
        else:
            return False

    else:
        return False

df["Check_3"] = df.apply(lambda x: check_3(name = x.Name_CS, distance = x.Distance), axis=1)




def check_4(name):

    if name > 0.99:
        return True
    else:
        return False

df["Check_4"] = df.apply(lambda x: check_4(name = x.Name_CS), axis=1)





rows = df.index[(df['Check_1'] == True) | (df['Check_2'] == True) | (df['Check_3'] == True) | (df['Check_4'] == True)].tolist()
false_rows = df.index[(df['Check_1'] == False) & (df['Check_2'] == False) & (df['Check_3'] == False) | (df['Check_4'] == True)].tolist()

df_true = df.loc[rows]
df_false = df.loc[false_rows]

df_true.to_csv('outputs/duplicates_CIHI.csv', index=False)
df_false.to_csv('outputs/NOT_duplicates_CIHI.csv', index=False)

hashes = df_true['to_remove'].tolist()

final = final[~final['idx'].isin(hashes)]
final.to_csv('deduplicated_CIHI.csv', index=False)
final.to_csv('inputs/deduplicated_CIHI.csv', index=False)




In [ ]:
#9_creat_pairs_Prov

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import json
import pandas as pd
import re
from rl_helper import strip_accents, AddressClean, haversine
import recordlinkage as rl
from recordlinkage.preprocessing import clean
import numpy as np
import math

'''
Read in source file, data file, and rename data file columns
'''
sourcefile = "/home/jovyan/data-vol-1/ODHF/LODE-ECDO/scripts/HealthFacilities/V2/7-Deduplication/inputs/deduplicated_CIHI.json"
with open(sourcefile) as source_f:
    Source = json.load(source_f)
    

df = pd.read_csv(Source["filename"],
               encoding=Source["encoding"],
               index_col=Source["index"])

print('I. Preprocessing - renaming columns, removing accents, and making string replacements.')

# reduce database to only the columns we use for comparisons
df = df[Source["column_map"].values()]
column_map = {val: key for key, val in Source["column_map"].items()}
df = df.rename(columns = column_map)

# remove accents

text_cols = ['Name','Address','StreetName','City']

for col in text_cols:
    df.loc[~df[col].isnull(),col]=df.loc[~df[col].isnull(),col].apply(strip_accents)
    
#make names and addresses lowercase
df['Name'] = df['Name'].apply(str)
df['Name'] = df['Name'].str.lower()

df['Address'] = df['Address'].apply(str)
df['Address'] = df['Address'].str.lower()

df['StreetName'] = df['StreetName'].apply(str)
df['StreetName'] = df['StreetName'].str.lower()

#apply text swaps in the Name column    
for swap in Source["text_map"]:
    start = r'\b'+re.escape(swap[0])+r'\b'
    df["Name"] = df["Name"].str.replace(start,swap[1], regex=True)




#standardise addresses using "AddressClean" function in the rl_helper module

df.loc[(~df['StreetName'].isnull()) & (df.Province != 'qc'), 'StreetName'] = df.loc[(~df['StreetName'].isnull()) & (df.Province != 'qc'), 'StreetName'].apply(AddressClean, args = ('en',))
df.loc[(~df['StreetName'].isnull()) & (df.Province == 'qc'), 'StreetName'] = df.loc[(~df['StreetName'].isnull()) & (df.Province == 'qc'), 'StreetName'].apply(AddressClean, args = ('fr',))

df.loc[(~df['Address'].isnull()) & (df.Province != 'qc'), 'Address'] = df.loc[(~df['Address'].isnull()) & (df.Province != 'qc'), 'Address'].apply(AddressClean, args = ('en',))
df.loc[(~df['Address'].isnull()) & (df.Province == 'qc'), 'Address'] = df.loc[(~df['Address'].isnull()) & (df.Province == 'qc'), 'Address'].apply(AddressClean, args = ('fr',))

#remove periods, apostrophes, commas, and hypens in the Name and address columns

r_list = [r".", r",", r"'", r"-"]

for r in r_list:

    df["Name"] = df["Name"].str.replace(r, ' ', regex=False)
    df["Address"] = df["Address"].str.replace(r, ' ', regex=False)

#remove excess whitespace
df["Name"] = df["Name"].str.replace(r" +", " ", regex=True)
df["Address"] = df["Address"].str.replace(r" +", " ", regex=True)

#standardise postal codes - just remove empty space and make sure it's all lower case

df.loc[~df.PostalCode.isnull(), 'PostalCode'] = df.loc[~df.PostalCode.isnull(), 'PostalCode'].str.replace(' ', '', regex=True).str.lower()

#create an extra temporary Name column with an additional level of cleaning

df['NameClean'] = clean(df["Name"])

#Some records have street number and street name, but no address field filled

df.loc[(df.Address.isnull())&\
       (~df.StreetName.isnull()),'Address']\
    =clean(df.loc[(df.Address.isnull())&\
       (~df.StreetName.isnull()),'StreetNumber']+' '+\
           df.loc[(df.Address.isnull())&\
       (~df.StreetName.isnull()),'StreetName']+' '+\
        df.loc[(df.Address.isnull())&\
       (~df.StreetName.isnull()),'City'])

#df.to_csv('test.csv')

r"""
II. Record Linkage


This is the section that uses the record linkage package to determine candidate pairs,
which will be evaluated separately.
"""
print('II. Record linkage - Now creating multiindex and performing comparisons')

indexer = rl.Index()
indexer.block('Province')
candidate_links = indexer.index(df)

print('Computing metrics for {} candidate pairs'.format(len(candidate_links)))

# likely to be a lot of records to match, so split into chunks
n = math.ceil(len(candidate_links) / 1E5)
chunks = rl.index_split(candidate_links, n)

# Comparison step
results = []

# n_jobs specifies number of cores for running in parallel
compare = rl.Compare(n_jobs=4)

compare.exact('StreetNumber', 'StreetNumber', label='StrNum_Match')
compare.exact('PostalCode', 'PostalCode', label='PC_Match')
compare.exact('FileName', 'FileName', label='File_Match')
compare.exact('Type', 'Type', label='Type_Match')
compare.string('Address', 'Address', method='damerau_levenshtein', label='Addr_DL')
compare.string('Address', 'Address', method='cosine', label='Addr_CS')
compare.string('Address', 'Address', method='damerau_levenshtein', label='StrName_DL')
compare.string('Address', 'Address', method='cosine', label='StrName_CS')
compare.string('City', 'City', method='damerau_levenshtein', label='City_DL')
compare.string('Name', 'Name', method='damerau_levenshtein', label='Name_DL')
compare.string('Name', 'Name', method='cosine', label='Name_CS')
compare.string('Name', 'Name', method='qgram', label='Name_Q')
compare.string("NameClean", "NameClean", method='damerau_levenshtein', label="CleanName_DL")


i = 0
for chunk in chunks:
    i += 1
    print('processing chunk {} of {}'.format(i,n))

    features = compare.compute(chunk, df)

    #reduce comparison matrix to entries where the name score is reasonably high

    cutoff = 0.5
    features = features.loc[features.Name_CS > cutoff]
    results.append(features)
f = pd.concat(results)
print('Score cut-off of {} reduced candidate pairs to {}'.format(cutoff, len(f)))

f['idx1'] = f.index.get_level_values(0)
f['idx2'] = f.index.get_level_values(1)

print('Merging on original dataframe and computing distance.')
f=f.merge(df, left_on='idx1', how='left', right_on='idx')

f=f.merge(df, left_on='idx2', how='left', right_on='idx', suffixes=('_1','_2'))

#add Haversine distance to pairs

f['Distance']=np.nan
f[['Latitude_1', 'Latitude_2', 'Longitude_1', 'Longitude_2']] = f[['Latitude_1', 'Latitude_2', 'Longitude_1', 'Longitude_2']].astype(float)
f.loc[(~f.Latitude_1.isnull())&(~f.Latitude_2.isnull()),'Distance']=f.loc[(~f.Latitude_1.isnull())&(~f.Latitude_2.isnull())].apply(lambda row: haversine(row), axis=1)

f=f[['idx1',
     'idx2',
     'FileName_1',
     'FileName_2',
     'File_Match',
     'Name_1',
     'Name_2',
     'Name_DL',
     'Name_CS',
     'Name_Q',
     'CleanName_DL',
     'Type_1',
     'Type_2',
     'Type_Match',
     'Address_1',
     'Address_2',
     'Addr_DL',
     'Addr_CS',
     'StrNum_Match',
     'StrName_DL',
     'StrName_CS',
     'PostalCode_1',
     'PostalCode_2',
     'PC_Match',
     'City_1',
     'City_2',
     'City_DL',
     'CSDUID_1',
     'CSDUID_2',
     'Distance']]



f.to_csv('outputs/pairs_PROV.csv'.format(Source["output_name"]),index=False,encoding='cp1252')



